In [31]:
import psycopg2
import pandas as pd
import scipy.stats as stats
import warnings

In [18]:
connection = psycopg2.connectconnection = psycopg2.connect(host="localhost", database="analysisdb", user="postgres",password="setouchi1311")

In [19]:
df = pd.read_sql("SELECT gridcode FROM trakan2204 INNER JOIN trakanmap ON trakan2204.計測地点番号 = trakanmap.地点番号 WHERE trakanmap.gridcode != '0' GROUP BY gridcode", con=connection )

C:\Users\BIGDATA_DEV\AppData\Local\Temp\ipykernel_8448\1600466108.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT gridcode FROM trakan2204 INNER JOIN trakanmap ON trakan2204.計測地点番号 = trakanmap.地点番号 WHERE trakanmap.gridcode != '0' GROUP BY gridcode", con=connection )


In [20]:
df.count()

gridcode    207
dtype: int64

In [38]:
nCt = 0
warnings.simplefilter('ignore')
for row in df.values:
    df2 = pd.read_sql("select 時刻,trakanmap.gridcode,断面交通量,case when stddev(t.断面交通量) over() = 0  then 0 else (t.断面交通量 - avg(t.断面交通量) over()) / stddev(t.断面交通量) over() end as zscore from trakan2204 AS t INNER JOIN trakanmap ON t.計測地点番号 = trakanmap.地点番号 where trakanmap.gridcode = '"+str(row[0])+"' ORDER BY 断面交通量 desc", con=connection )
    df3over = df2[df2['zscore'] >= 3].count()
    if nCt == 0:
        df3=df2[df2['zscore'] < 3]
    else:
        df3=df3.append(df2[df2['zscore'] < 3], ignore_index=True)
    nCt = nCt + df3over.gridcode
print(str(nCt) + "除外")


4590除外


In [39]:
df3.head()

,時刻,gridcode,断面交通量,zscore
0,2022-04-08 07:20:00,50324526133,57,2.97371
1,2022-04-01 07:40:00,50324526133,57,2.97371
2,2022-04-30 11:25:00,50324526133,57,2.97371
3,2022-04-10 08:40:00,50324526133,57,2.97371
4,2022-04-14 11:25:00,50324526133,57,2.97371


In [43]:
df = df3.sort_values(['時刻', 'gridcode']).drop('zscore', axis=1)

In [44]:
connection.close()

In [45]:
df.head()

,時刻,gridcode,断面交通量
8278,2022-04-01,50324526133,1
16248,2022-04-01,50324526341,1
24033,2022-04-01,50324542441,4
33187,2022-04-01,50324542442,2
43560,2022-04-01,50324585213,1


In [46]:
df.to_csv("trakan2204_z3item.csv", index = False)

In [ ]:
#SELECT * FROM trakan2209_z3out WHERE  extract(hour FROM 時刻) = 9 ORDER BY 時刻,gridcode

In [15]:
df = pd.read_sql("SELECT yyyymmdd,start_mesh_area,start_time,end_mesh_area,end_time, cast(REGEXP_REPLACE(population,'[^.0-9]+','', 'g') AS FLOAT) as population   FROM kddi_location where odroutetype = 'K' and trim(translate(population, '.0123456789', ' ')) is not null AND population != ''", con=connection )

C:\Users\BIGDATA_DEV\AppData\Local\Temp\ipykernel_8216\2294289007.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT yyyymmdd,start_mesh_area,start_time,end_mesh_area,end_time, cast(REGEXP_REPLACE(population,'[^.0-9]+','', 'g') AS FLOAT) as population   FROM kddi_location where odroutetype = 'K' and trim(translate(population, '.0123456789', ' ')) is not null AND population != ''", con=connection )


In [16]:
df.count()

yyyymmdd           228386
start_mesh_area    228386
start_time         228386
end_mesh_area      228386
end_time           228386
population         228386
dtype: int64

In [17]:
df['zscore'] = stats.zscore(df['population'])

In [18]:
df.head()

,yyyymmdd,start_mesh_area,start_time,end_mesh_area,end_time,population,zscore
0,20211001,50323475431,11:30:00,50325641432,13:30:00,30.82,-0.729910
1,20211001,50323475443,11:30:00,50323486123,12:00:00,74.89,1.268918
2,20211001,50323475443,14:30:00,50325526414,15:00:00,50.13,0.145910
3,20211001,50323486141,13:00:00,50324510341,13:30:00,34.60,-0.558465
4,20211001,50323566232,11:00:00,50323563232,11:30:00,55.94,0.409427


In [19]:
df[df['zscore'] >= 3].count()

yyyymmdd           4161
start_mesh_area    4161
start_time         4161
end_mesh_area      4161
end_time           4161
population         4161
zscore             4161
dtype: int64

In [20]:
outdf = df[df['zscore'] < 3].drop('zscore', axis=1)
outdf.to_csv("kddi_location_out_K.csv", index = False)

In [85]:
outdf

,yyyymmdd,start_mesh_area,start_time,end_mesh_area,end_time,population
19,20211001,50325546322,11:45:00,50325546433,12:30:00,17542
22,20211001,50325546323,08:30:00,50326620132,09:15:00,23491
67,20211001,50325546411,16:00:00,50326579211,17:45:00,17542
100,20211001,50325546433,13:15:00,50325546411,13:45:00,17542
169,20211001,50325547312,20:45:00,50325516142,21:30:00,18545
...,...,...,...,...,...,...
1434990,20211006,50325681412,15:45:00,50324693322,17:00:00,23633
1434995,20211008,50327631232,13:30:00,50326650332,15:30:00,17542
1435034,20220417,50325620311,13:15:00,50325680312,14:00:00,35027
1435230,20211026,50325671141,13:00:00,50325539123,13:45:00,20824
